<a href="https://colab.research.google.com/github/romerofernandes/Car_consumption/blob/main/Car_consumption.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Importações

In [135]:
import pandas as pd
import numpy as np
import seaborn as sns
import glob
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from  sklearn.preprocessing import OneHotEncoder as OHE
from sklearn.compose import make_column_transformer
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_regression, mutual_info_regression

In [137]:
#Modelos
from sklearn.svm import SVR
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor

In [138]:
#Análise dos modelos
from sklearn.metrics import r2_score
from sklearn.metrics import mean_absolute_percentage_error as MAPE
from sklearn.metrics import mean_squared_error as MSE

In [59]:
pd.set_option('display.max_rows', None)

In [60]:
all_csv_files = glob.glob("/content/*.csv")
all_csv_files

['/content/unclean focus.csv',
 '/content/ford.csv',
 '/content/unclean cclass.csv',
 '/content/merc.csv',
 '/content/skoda.csv',
 '/content/focus.csv',
 '/content/audi.csv',
 '/content/bmw.csv',
 '/content/vw.csv',
 '/content/vauxhall.csv',
 '/content/hyundi.csv',
 '/content/cclass.csv',
 '/content/toyota.csv']

In [61]:
df_car = pd.concat(map(pd.read_csv, all_csv_files), axis=0, ignore_index=True)

In [62]:
df_car.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 118150 entries, 0 to 118149
Data columns (total 16 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   model         117995 non-null  object 
 1   year          117903 non-null  float64
 2   price         117995 non-null  object 
 3   transmission  117995 non-null  object 
 4   mileage       117077 non-null  object 
 5   fuel type     3517 non-null    object 
 6   engine size   9345 non-null    object 
 7   mileage2      9399 non-null    object 
 8   fuel type2    8537 non-null    object 
 9   engine size2  8537 non-null    object 
 10  reference     9455 non-null    object 
 11  fuelType      108540 non-null  object 
 12  tax           94327 non-null   float64
 13  mpg           99187 non-null   float64
 14  engineSize    108540 non-null  float64
 15  tax(£)        4860 non-null    float64
dtypes: float64(5), object(11)
memory usage: 14.4+ MB


#Transformações

In [63]:
df_car.dropna(subset=['mpg'], how='any', inplace=True)

In [64]:
df_car.dropna(axis='columns', inplace=True)

In [65]:
df_car.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 99187 entries, 5604 to 118149
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   model         99187 non-null  object 
 1   year          99187 non-null  float64
 2   price         99187 non-null  object 
 3   transmission  99187 non-null  object 
 4   mileage       99187 non-null  object 
 5   fuelType      99187 non-null  object 
 6   mpg           99187 non-null  float64
 7   engineSize    99187 non-null  float64
dtypes: float64(3), object(5)
memory usage: 6.8+ MB


##Combinando modelos

In [66]:
a = df_car.model.value_counts().reset_index()

In [67]:
a.columns = ['model', 'qtd']

In [68]:
finish = False
limit_s = 3000
limit_i = 2000

group_car = 1
find_start = 0
while a.qtd.iloc[find_start] > limit_i:
  find_start +=1

previous = find_start
actual = find_start + 2

while finish is False:
  if (a.qtd.iloc[previous:actual].sum() < limit_s) & (a.qtd.iloc[previous:actual].sum() >= limit_i):
    modelos = a.model.iloc[previous:actual].values
    
    for modelo in modelos:
      df_car.model.replace(modelo, 'car' + str(group_car), inplace=True)
    
    group_car +=1
    previous = actual
    actual += 2
  else:
    if (a.qtd.iloc[previous:a.shape[0]].sum() < limit_s) & (a.qtd.iloc[previous:a.shape[0]].sum() >= limit_i):
      modelos = a.model.iloc[previous:a.shape[0]].values
      for modelo in modelos:
        df_car.model.replace(modelo, 'car' + str(group_car), inplace=True)
      finish=True

    elif a.qtd.iloc[previous:actual].sum() <= limit_i:
      actual +=1
    elif a.qtd.iloc[previous:actual].sum() > limit_s:
      previous +=1
      actual +=1
  
  if actual > a.shape[0]:
    finish=True

##Combinando anos

In [69]:
df_car.year = df_car.year.astype(int)
df_car.year = df_car.year.astype(str)

In [70]:
df_car.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 99187 entries, 5604 to 118149
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   model         99187 non-null  object 
 1   year          99187 non-null  object 
 2   price         99187 non-null  object 
 3   transmission  99187 non-null  object 
 4   mileage       99187 non-null  object 
 5   fuelType      99187 non-null  object 
 6   mpg           99187 non-null  float64
 7   engineSize    99187 non-null  float64
dtypes: float64(2), object(6)
memory usage: 6.8+ MB


In [71]:
df_car.loc[df_car.year=='2060'].index

Int64Index([23330], dtype='int64')

In [72]:
df_car.drop(index=df_car.loc[df_car.year=='2060'].index, inplace=True)

In [73]:
b = df_car.year.value_counts().reset_index()

In [74]:
b.columns = ['year', 'qtd']

In [75]:
anos = b.year.iloc[4:].values
for ano in anos:
        df_car.year.replace(ano, 'outros', inplace=True)

In [76]:
df_car.year.value_counts()

2019      26771
2017      21733
outros    21168
2016      15850
2018      13664
Name: year, dtype: int64

##Combinando Engine Size

In [77]:
df_car.head()

,model,year,price,transmission,mileage,fuelType,mpg,engineSize
5604,Fiesta,2017,12000,Automatic,15944,Petrol,57.7,1.0
5605,Focus,2018,14000,Manual,9083,Petrol,57.7,1.0
5606,Focus,2017,13000,Manual,12456,Petrol,57.7,1.0
5607,Fiesta,2019,17500,Manual,10460,Petrol,40.3,1.5
5608,Fiesta,2019,16500,Automatic,1482,Petrol,48.7,1.0


In [78]:
c = df_car.engineSize.value_counts().reset_index()

In [79]:
c.columns = ['engineSize', 'qtd']

In [80]:
c.qtd.iloc[7:].sum()

10939

In [81]:
engine_size = c.engineSize.iloc[7:].values
df_car.engineSize.replace(engine_size, 'outros', inplace=True)

In [82]:
c

,engineSize,qtd
0,2.0,27233
1,1.0,17323
2,1.5,11418
3,1.4,11026
4,1.6,8837
5,1.2,6963
6,3.0,5447
7,2.1,4018
8,1.8,1717
9,1.3,1372


In [83]:
c = df_car.engineSize.value_counts().reset_index()
c.columns = ['engineSize', 'qtd']

In [84]:
c

,engineSize,qtd
0,2.0,27233
1,1.0,17323
2,1.5,11418
3,1.4,11026
4,outros,10939
5,1.6,8837
6,1.2,6963
7,3.0,5447


In [85]:
c.qtd.iloc[6:].sum()

12410

In [86]:
df_car.engineSize.replace([1.6, 1.2], '1.2/1.6', inplace=True)

In [87]:
df_car.engineSize.replace(3.0, 'outros', inplace=True)

In [88]:
df_car.engineSize.value_counts()

2.0        27233
1.0        17323
outros     16386
1.2/1.6    15800
1.5        11418
1.4        11026
Name: engineSize, dtype: int64

In [89]:
df_car.price = df_car.price.astype('float')
df_car.mileage = df_car.mileage.astype('float')

In [90]:
df_car.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 99186 entries, 5604 to 118149
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   model         99186 non-null  object 
 1   year          99186 non-null  object 
 2   price         99186 non-null  float64
 3   transmission  99186 non-null  object 
 4   mileage       99186 non-null  float64
 5   fuelType      99186 non-null  object 
 6   mpg           99186 non-null  float64
 7   engineSize    99186 non-null  object 
dtypes: float64(3), object(5)
memory usage: 6.8+ MB


#testes

In [91]:
df_car.head()

,model,year,price,transmission,mileage,fuelType,mpg,engineSize
5604,Fiesta,2017,12000.0,Automatic,15944.0,Petrol,57.7,1.0
5605,Focus,2018,14000.0,Manual,9083.0,Petrol,57.7,1.0
5606,Focus,2017,13000.0,Manual,12456.0,Petrol,57.7,1.0
5607,Fiesta,2019,17500.0,Manual,10460.0,Petrol,40.3,1.5
5608,Fiesta,2019,16500.0,Automatic,1482.0,Petrol,48.7,1.0


In [92]:
df_car.engineSize = df_car.engineSize.astype(str)

In [119]:
y = df_car.mpg

In [128]:
X = df_car.drop(columns='mpg')#, inplace=True)

In [129]:
features_cont = ['price', 'mileage']
features_categ = ['model', 'year', 'transmission', 'fuelType', 'engineSize']

In [130]:
transformer = make_column_transformer((OHE(drop='first', sparse=False), features_categ),
                                      #(StandardScaler(), features_cont),
                                      remainder='passthrough')

In [131]:
X = transformer.fit_transform(X)

In [132]:
X.shape

(99186, 55)

In [133]:
print(X)

[[0.0000e+00 0.0000e+00 0.0000e+00 ... 0.0000e+00 1.2000e+04 1.5944e+04]
 [0.0000e+00 0.0000e+00 0.0000e+00 ... 0.0000e+00 1.4000e+04 9.0830e+03]
 [0.0000e+00 0.0000e+00 0.0000e+00 ... 0.0000e+00 1.3000e+04 1.2456e+04]
 ...
 [0.0000e+00 0.0000e+00 0.0000e+00 ... 0.0000e+00 4.9950e+03 4.6000e+04]
 [0.0000e+00 0.0000e+00 0.0000e+00 ... 1.0000e+00 3.9950e+03 6.0700e+04]
 [0.0000e+00 0.0000e+00 0.0000e+00 ... 1.0000e+00 4.4950e+03 4.5128e+04]]


#Preparação

In [144]:
#Separação dos dados para treino e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.15, random_state=123)

###Random Forest Regressor

####Treinamento, validação e teste

In [149]:
#Criação do Modelo Random Forest
RFR = RandomForestRegressor()#n_estimators= 185, max_depth=39)

In [150]:
#Treinamento do Modelo Random Forest
RFR.fit(X_train, y_train)

RandomForestRegressor()

In [151]:
#Criação das previsões do Modelo Random Forest
y_predict_RFR = RFR.predict(X_test)

In [153]:
RFR.score(X_test, y_test)

0.8908974771393355